In [2]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [3]:
from google.colab import files
import whisper
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np


In [4]:
# Upload the audio file
uploaded = files.upload()
path = next(iter(uploaded))


Saving hate.mp3 to hate.mp3


In [6]:
# Parameters
num_speakers = int(input("Enter the number of speakers: "))  # Set the number of speakers
language = 'English'  # Set the language
model_size = 'base'  # Set the Whisper model size

Enter the number of speakers:  2


In [7]:
# Custom speaker labels
speaker_labels = []
for i in range(num_speakers):
    label = input(f"Enter label for speaker {i + 1}: ")
    speaker_labels.append(label)


Enter label for speaker 1: Tony Stark
Enter label for speaker 2: Pepper Pots


In [8]:
# Adjust model name based on language and size
model_name = model_size
if language == 'English' and model_size != 'large':
    model_name += '.en'


In [9]:
# Install required packages
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

In [10]:
# Load Whisper model
model = whisper.load_model(model_size)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 160MiB/s]


In [11]:
# Function to convert segment to embedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))


In [12]:
# Convert audio to wav format if needed
if path[-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
    path = 'audio.wav'

In [13]:
# Transcribe audio
result = model.transcribe(path)
segments = result["segments"]

In [14]:
# Get audio duration
with contextlib.closing(wave.open(path, 'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)

In [15]:
# Initialize audio object
audio = Audio()

In [16]:
# Function to extract segment embedding
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)
    return embedding_model(waveform[None])


In [17]:
# Create embeddings for each segment
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [18]:
# Perform clustering to identify speakers
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
    segments[i]["speaker"] = speaker_labels[labels[i]]

In [19]:
# Function to format time
def time(secs):
    return datetime.timedelta(seconds=round(secs))


In [20]:
# Write transcript with speaker labels to file
with open("transcript.txt", "w") as f:
    for i, segment in enumerate(segments):
        if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
            f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
        f.write(segment["text"][1:] + ' ')

print("Transcript with speaker labels saved to transcript.txt")

Transcript with speaker labels saved to transcript.txt
